In [13]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import (
    Input, RandomFlip, RandomBrightness, RandomContrast, RandomZoom, Rescaling,
    Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, Lambda
)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras import backend as k

In [14]:
# --- PARÂMETROS ---
dataset_dir = 'dataset/'
img_size = 224
seed = 1000
n_splits = 5
batch_size = 16
epochs = 100

In [15]:
image_paths = []
labels = []
class_names = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])
class_map = {name: i for i, name in enumerate(class_names)}

print(f"Classes encontradas: {class_map}")

for class_name, class_idx in class_map.items():
    class_dir = os.path.join(dataset_dir, class_name)
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith(('.png', '.jpg')):
            image_paths.append(os.path.join(class_dir, img_name))
            labels.append(class_idx)

image_paths = np.array(image_paths)
labels = np.array(labels)

Classes encontradas: {'bleached_corals': 0, 'healthy_corals': 1}


In [18]:
def create_model():
    k.clear_session()
    data_augmentation = Sequential([
        RandomFlip(mode="horizontal", seed=seed),
        RandomBrightness(factor=0.2, seed=seed),
        RandomContrast(factor=0.2, seed=seed),
        RandomZoom(0.2, seed=seed)
    ])
    
    base_model = tf.keras.applications.VGG19(weights='imagenet', include_top=False)
    base_model.trainable = False  # Congela a VGG19

    model = Sequential([
        Input(shape=(img_size, img_size, 3)),
        data_augmentation,
        Lambda(preprocess_input),
        base_model,
        Flatten(),
        Dense(units=832, activation="relu", kernel_initializer="he_normal"),
        Dropout(0.4),
        Dense(units=320, activation="relu", kernel_initializer="he_normal"),
        Dropout(0.3),
        Dense(units=704, activation="relu", kernel_initializer="he_normal"),
        Dense(units=1, activation="sigmoid")
    ])

    optimizer = Adam(learning_rate=1e-4)

    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=[
            BinaryAccuracy(name='accuracy'),
            Precision(name='precision'),
            Recall(name='recall')
        ]
    )
    
    return model

In [19]:
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, [img_size, img_size])
    image.set_shape([img_size, img_size, 3])
    return image, label

In [20]:
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

val_loss_per_fold = []
val_accuracy_per_fold = []
val_precision_per_fold = []
val_recall_per_fold = []
f1_per_fold = []
total_cm = np.zeros((len(class_names), len(class_names)))

In [21]:
# Configura o layout da figura para os subplots dos folds
# Calcule o número de linhas e colunas para os subplots
n_cols = 3 # Número de colunas desejadas, ajuste conforme preferir
n_rows = (n_splits + n_cols - 1) // n_cols # Garante linhas suficientes
plt.figure(figsize=(n_cols * 6, n_rows * 5)) # Ajuste o tamanho da figura conforme necessário

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

In [22]:
for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")

    train_paths, val_paths = image_paths[train_idx], image_paths[val_idx]
    train_labels, val_labels = labels[train_idx], labels[val_idx]

    train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))

    train_ds = (train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
                .shuffle(buffer_size=len(train_paths))
                .batch(batch_size)
                .prefetch(tf.data.AUTOTUNE))
    val_ds = (val_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
              .batch(batch_size)
              .prefetch(tf.data.AUTOTUNE))

    model = create_model()
    checkpoint_path = f"./checkpoint/vgg-19/best_model_fold_{fold+1}.keras"

    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        mode='max', # 'max' para acurácia, 'min' para perda
        restore_best_weights=True,
        verbose=1
    )

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[early_stopping],
        verbose=1
    )

    model.save(checkpoint_path)

    best_epoch_index = np.argmax(history.history['val_accuracy']) if 'val_accuracy' in history.history else -1
    if best_epoch_index == -1: # fallback para perda se acurácia não estiver disponível ou for monitorada
        best_epoch_index = np.argmin(history.history['val_loss'])

    val_loss = history.history['val_loss'][best_epoch_index]
    val_acc = history.history['val_accuracy'][best_epoch_index]
    val_prec = history.history.get('val_precision', [None])[best_epoch_index]
    val_rec = history.history.get('val_recall', [None])[best_epoch_index]

    val_loss_per_fold.append(val_loss)
    val_accuracy_per_fold.append(val_acc)
    val_precision_per_fold.append(val_prec)
    val_recall_per_fold.append(val_rec)

    f1 = 2 * (val_prec * val_rec) / (val_prec + val_rec + 1e-7) if val_prec is not None and val_rec is not None else None
    f1_per_fold.append(f1)

    print(f"Score for fold {fold+1}: Loss={val_loss:.4f}; Accuracy={val_acc:.4f}; Precision={val_prec:.4f}; Recall={val_rec:.4f}; F1-Score={f1:.4f}")

    y_pred_probs = model.predict(val_ds)
    y_pred = (y_pred_probs > 0.5).astype("int32").flatten()

    y_true_list = []
    for _, y_batch in val_ds:
        y_true_list.append(y_batch.numpy())
    y_true = np.concatenate(y_true_list, axis=0)


    # Gráfico de Confusão para o fold atual como um subplot
    cm = confusion_matrix(y_true, y_pred)
    total_cm += cm # Acumula a matriz de confusão

    plt.subplot(n_rows, n_cols, fold + 1) # (linhas, colunas, índice do subplot)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Previsto')
    plt.ylabel('Verdadeiro')
    plt.title(f'Fold {fold+1}')


--- Fold 1/5 ---
Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 88s 1s/step - accuracy: 0.5894 - loss: 3.1792 - precision: 0.5598 - recall: 0.5690 - val_accuracy: 0.7589 - val_loss: 1.2913 - val_precision: 0.7302 - val_recall: 0.7731
Epoch 2/100
 4/64 ━━━━━━━━━━━━━━━━━━━━ 1:08 1s/step - accuracy: 0.7591 - loss: 1.7616 - precision: 0.6968 - recall: 0.8810

KeyboardInterrupt: 